# Imports

In [25]:
!pip install --upgrade --force-reinstall nltk

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl.metadata (41 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl (273 kB)
Using cached click-8.2.1-py3-none-any.whl (102 kB)
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)
  Attempting uninstall: regex
    Found existing installation: regex 2024.11.6
    Uninstalling regex-2024.11.6:
      Successfully uninstalled regex-2024.11.6
  Attempting uninstall: joblib
    Found existing installation: joblib 1.5.1
    Uninstalling joblib-1.5.1:
      Successfully 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.5.1 requires keyring>=17.0.0, but you have keyring 8.7 which is incompatible.


In [26]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\k\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [27]:
import os
import re
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import torch
from torch.nn.utils.rnn import pad_sequence

#

# Text processing

In [28]:
def file_to_sentence_list(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()


    sentences = [sentence.strip() for sentence in re.split(
        r'(?<=[.!?])\s+', text) if sentence.strip()]

    return sentences

file_path = '../data/raw/data.txt'
text_data = file_to_sentence_list(file_path)

In [29]:
len(text_data)

6216

In [30]:
def my_tokenizer(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = text.split()
    return tokens

tokenized_sentences = [my_tokenizer(sentence) for sentence in text_data]
tokenized_sentences

[['homa', 'mkhbbyin', 'chi', 'haja', 'ana', 'mti99en'],
 ['bayna', 'homa', 'tay7awlo', 'ib9aw', 'mbrrdin'],
 ['lotilat', 'mabaynach', 'fihom', 'mori7in', 'bzzaf'],
 ['ghaliban', 'ghayjrriw', '3lih', 'mn', 'lkhdma'],
 ['tab3an', 'rah', 'mkta2eb'],
 ['tayballia', 'ghanmchi'],
 ['ara', 'lia', 'dak', 'sac'],
 ['ghanmred'],
 ['knt',
  'dima',
  '3arf',
  'annaha',
  'bghatna',
  'nmoutou',
  'bghit',
  'n3ref',
  'ch7al',
  'b9a',
  'lia',
  'tlwe9t',
  'bach',
  'n9ra',
  'ghaykon',
  '3ndna',
  'mti7an',
  'ala',
  'hadchi',
  'lli',
  '9sedti',
  'ssimana',
  'jjaya',
  'nhar',
  'lkhmis',
  'ghanghttiw',
  'ga3',
  'lmawad',
  'tal',
  'tlat',
  'jjay',
  'ewa',
  'bda',
  'l9raya',
  'bkri'],
 ['wakha',
  'ala',
  '3ndi',
  'ay',
  'as2ila',
  'wach',
  'nsifthom',
  'lik',
  'f',
  'email'],
 ['walakin',
  'ghanjawb',
  '3lihom',
  'fl',
  'aw9at',
  't',
  'lkhdma',
  'idan',
  '3afak',
  'matsifthomch',
  'm3ttel',
  'nhar',
  'larb3',
  'bl3chia',
  'chokran',
  'a',
  'lalla',
  '

In [31]:
import string
from nltk.tokenize import word_tokenize

def clean_and_tokenize(sentence):
    sentence = sentence.lower()
    tokens = word_tokenize(sentence)
    # Remove punctuation tokens
    tokens = [token for token in tokens if token not in string.punctuation]
    return tokens


In [65]:
import re
from collections import defaultdict, Counter

def preprocess_text(text):
    """Normalize and prepare Darija text for BPE"""
    # Remove Arabic diacritics
    text = re.sub(r'[ًٌٍَُِّْ]', '', text)
    # Normalize alef variants
    text = re.sub(r'[إأآا]', 'ا', text)
    # Convert ta-marbuta to ha
    text = re.sub(r'ة', 'ه', text)
    
    # Normalize Romanized variations
    replacements = {
        r'[éèêë]': 'e',
        r'[àâä]': 'a',
        r'[îï]': 'i',
        r'[ôö]': 'o',
        r'[ûüù]': 'u',
        r'ch': 'sh',
        r'9': 'q',
        r'5': 'kh',
    }
    for pattern, repl in replacements.items():
        text = re.sub(pattern, repl, text)
    
    return text.lower()

def get_vocab(text):
    """Extract word frequencies from text"""
    words = re.findall(r'\w+', text)
    return Counter(words)

def get_stats(vocab):
    """Count pairs of adjacent symbols in vocab"""
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[(symbols[i], symbols[i+1])] += freq
    return pairs

def merge_vocab(pair, vocab):
    """Apply a merge operation to the vocabulary"""
    v_out = {}
    bigram = re.escape(' '.join(pair))
    pattern = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    for word in vocab:
        # Merge the best pair in the word
        w_out = pattern.sub(''.join(pair), word)
        v_out[w_out] = vocab[word]
    return v_out

def train_bpe(text, num_merges=20, min_freq=100):
    """Train BPE on Darija text"""
    text = preprocess_text(text)
    vocab = get_vocab(text)
    # Initialize vocab keys as sequences of chars separated by spaces
    vocab = {' '.join(word): freq for word, freq in vocab.items()}
    
    merges = []
    for i in range(num_merges):
        pairs = get_stats(vocab)
        if not pairs:
            print("No more pairs to merge.")
            break
        best_pair = max(pairs, key=pairs.get)
        if pairs[best_pair] < min_freq:
            print(f"Stopping early at merge {i+1} due to low frequency: {pairs[best_pair]} < {min_freq}")
            break
        merges.append(best_pair)
        vocab = merge_vocab(best_pair, vocab)
    return merges

def apply_bpe(text, merges):
    """Segment text using learned BPE rules"""
    text = preprocess_text(text)
    words = re.findall(r'\w+', text)
    merges_set = set(merges)
    segmented_words = []
    
    for word in words:
        word_chars = list(word)
        i = 0
        while i < len(word_chars) - 1:
            pair = (word_chars[i], word_chars[i+1])
            if pair in merges_set:
                # Merge pair tokens
                word_chars[i] = word_chars[i] + word_chars[i+1]
                del word_chars[i+1]
                # After merge, check again from current position
            else:
                i += 1
        segmented_words.append(word_chars)
    return segmented_words


text = '\n'.join(text_data)
merges = train_bpe(text , num_merges=2000, min_freq=1)

print("Learned merges:")
for i, (a, b) in enumerate(merges):
    print(f"{i+1}. {a} + {b} -> {a+b}")

segmented = apply_bpe(text, merges)

print("\nSegmented words (first 10):")
for word_tokens in segmented[:10]:
    print(word_tokens)


Learned merges:
1. s + h -> sh
2. h + a -> ha
3. m + a -> ma
4. l + i -> li
5. o + u -> ou
6. l + a -> la
7. a + n -> an
8. d + i -> di
9. r + a -> ra
10. y + a -> ya
11. w + a -> wa
12. d + a -> da
13. k + h -> kh
14. t + a -> ta
15. n + a -> na
16. t + i -> ti
17. b + a -> ba
18. e + r -> er
19. k + a -> ka
20. sh + i -> shi
21. 3 + a -> 3a
22. g + h -> gh
23. g + ha -> gha
24. s + a -> sa
25. gh + i -> ghi
26. e + k -> ek
27. e + n -> en
28. ha + d -> had
29. n + i -> ni
30. r + i -> ri
31. e + l -> el
32. k + an -> kan
33. s + s -> ss
34. a + l -> al
35. f + i -> fi
36. k + ha -> kha
37. q + a -> qa
38. k + i -> ki
39. e + d -> ed
40. b + i -> bi
41. 3 + la -> 3la
42. gha + di -> ghadi
43. an + a -> ana
44. ka + y -> kay
45. j + a -> ja
46. e + m -> em
47. 7 + a -> 7a
48. e + b -> eb
49. f + a -> fa
50. b + ghi -> bghi
51. z + a -> za
52. m + 3a -> m3a
53. s + i -> si
54. li + k -> lik
55. l + m -> lm
56. wa + sh -> wash
57. w + i -> wi
58. ra + h -> rah
59. li + ya -> liya
60. k +

In [66]:
merges

[('s', 'h'),
 ('h', 'a'),
 ('m', 'a'),
 ('l', 'i'),
 ('o', 'u'),
 ('l', 'a'),
 ('a', 'n'),
 ('d', 'i'),
 ('r', 'a'),
 ('y', 'a'),
 ('w', 'a'),
 ('d', 'a'),
 ('k', 'h'),
 ('t', 'a'),
 ('n', 'a'),
 ('t', 'i'),
 ('b', 'a'),
 ('e', 'r'),
 ('k', 'a'),
 ('sh', 'i'),
 ('3', 'a'),
 ('g', 'h'),
 ('g', 'ha'),
 ('s', 'a'),
 ('gh', 'i'),
 ('e', 'k'),
 ('e', 'n'),
 ('ha', 'd'),
 ('n', 'i'),
 ('r', 'i'),
 ('e', 'l'),
 ('k', 'an'),
 ('s', 's'),
 ('a', 'l'),
 ('f', 'i'),
 ('k', 'ha'),
 ('q', 'a'),
 ('k', 'i'),
 ('e', 'd'),
 ('b', 'i'),
 ('3', 'la'),
 ('gha', 'di'),
 ('an', 'a'),
 ('ka', 'y'),
 ('j', 'a'),
 ('e', 'm'),
 ('7', 'a'),
 ('e', 'b'),
 ('f', 'a'),
 ('b', 'ghi'),
 ('z', 'a'),
 ('m', '3a'),
 ('s', 'i'),
 ('li', 'k'),
 ('l', 'm'),
 ('wa', 'sh'),
 ('w', 'i'),
 ('ra', 'h'),
 ('li', 'ya'),
 ('k', 'ou'),
 ('k', 'o'),
 ('n', 't'),
 ('d', 'er'),
 ('j', 'i'),
 ('ka', 't'),
 ('q', 'i'),
 ('ghi', 'r'),
 ('d', 'ya'),
 ('e', 't'),
 ('sh', 'a'),
 ('da', 'r'),
 ('di', 'al'),
 ('m', 'n'),
 ('e', '3'),
 ('da',

In [32]:
def generate_ngrams(tokenized_sentences, max_len=5):
    sequences = []
    for sentence in tokenized_sentences:
        for i in range(1,len(sentence)):
            context = sentence[max(0,i-max_len):i]
            target = sentence[i]
            sequences.append((context, target))
        
    return sequences

In [53]:
from collections import Counter
import random

cleaned_tokenized_sentences = [clean_and_tokenize(sentence) for sentence in text_data]

all_words = [word for sentence in cleaned_tokenized_sentences for word in sentence]
word_counts = Counter(all_words)
threshold = 5
indexs = []
for i, (word, count) in enumerate(word_counts.items()):
    if count <= threshold:
        indexs.append(i)
    
indexs_choosed = random.sample(indexs, 1000)
indexs_choosed=[]
vocab = [word for i, (word, count) in enumerate(word_counts.items()) if count > threshold]

word2idx = {'<PAD>': 0, '<UNK>': 1}
for idx, word in enumerate(sorted(vocab), start=2):
    word2idx[word] = idx

# Replace rare words in sentences with <UNK>
def replace_rare_words(sentence):
    return [word if word in word2idx else '<UNK>' for word in sentence]

processed_sentences = [replace_rare_words(sentence) for sentence in cleaned_tokenized_sentences]

# Invert the dictionary
idx2word = {idx: word for word, idx in word2idx.items()}


In [56]:
all_words_in_processed_sentences = [word for sentence in processed_sentences for word in sentence]
word_counts_in_processed_sentences = Counter(all_words_in_processed_sentences)

In [60]:
word_counts_in_processed_sentences

Counter({'<UNK>': 61020,
         'ghadi': 2826,
         'had': 2586,
         'ana': 2366,
         'chi': 2285,
         't': 2191,
         'li': 2163,
         '3la': 1952,
         'wach': 1924,
         'rah': 1906,
         'ghir': 1723,
         'f': 1633,
         'liya': 1480,
         'ou': 1461,
         'bach': 1341,
         'mn': 1264,
         'lik': 1218,
         'dial': 1209,
         '3lach': 1101,
         'mabghitich': 1081,
         'l': 990,
         'n': 957,
         'hadchi': 953,
         'kan': 948,
         '3emrek': 908,
         'm3a': 895,
         'bezzaf': 853,
         'koun': 853,
         'dyal': 843,
         'men': 818,
         'o': 747,
         '9dert': 710,
         'fik': 635,
         'bla': 627,
         'nta': 612,
         'walakin': 605,
         'daba': 605,
         'bzaf': 584,
         'ila': 560,
         'fin': 548,
         'dar': 547,
         'ma': 539,
         'bghit': 537,
         'hadi': 531,
         '3lik': 528,
       

In [67]:
# First, preprocess and tokenize sentences into words, then apply BPE on each word:
bpe_tokenized_sentences = []
for sentence in text_data:
    # Preprocess & split into words (you may already have a cleaner tokenizer)
    words = re.findall(r'\w+', preprocess_text(sentence))
    
    # Apply BPE on each word (returns list of subword tokens)
    bpe_tokens_in_sentence = []
    for word in words:
        word_chars = list(word)
        i = 0
        while i < len(word_chars) - 1:
            pair = (word_chars[i], word_chars[i+1])
            if pair in merges:
                word_chars[i] = word_chars[i] + word_chars[i+1]
                del word_chars[i+1]
            else:
                i += 1
        bpe_tokens_in_sentence.extend(word_chars)
    
    bpe_tokenized_sentences.append(bpe_tokens_in_sentence)


In [68]:
from collections import Counter

# Flatten list of tokens from all sentences
all_bpe_tokens = [token for sent in bpe_tokenized_sentences for token in sent]

bpe_token_counts = Counter(all_bpe_tokens)

# Optional threshold to filter rare tokens
threshold = 5
vocab = [token for token, count in bpe_token_counts.items() if count > threshold]

# Build vocab to index dictionary
word2idx = {'<PAD>': 0, '<UNK>': 1}
for idx, token in enumerate(sorted(vocab), start=2):
    word2idx[token] = idx


In [71]:
len(bpe_token_counts)

700

In [72]:
bpe_token_counts

Counter({'a': 29481,
         'i': 17588,
         'h': 11520,
         'n': 9317,
         'u': 9004,
         'k': 7925,
         'sh': 7500,
         'b': 7259,
         't': 6582,
         'l': 6463,
         'gh': 6247,
         'ya': 5918,
         'as': 5865,
         'li': 5571,
         'ad': 5365,
         'had': 5325,
         '3l': 5317,
         'shi': 4936,
         'al': 4441,
         'ou': 4436,
         'ka': 4383,
         'na': 4275,
         'q': 4197,
         'kh': 3805,
         'o': 3698,
         'ba': 3632,
         'd': 3613,
         'f': 3538,
         'r': 3472,
         's': 3464,
         'ha': 3388,
         'ar': 3332,
         'an': 3215,
         'ni': 3149,
         'de': 3148,
         'ghi': 3084,
         'ti': 3069,
         'at': 3059,
         '3': 3035,
         'nt': 2961,
         'ek': 2890,
         'm3': 2883,
         'ana': 2846,
         '7a': 2812,
         'ta': 2686,
         'ra': 2650,
         'ay': 2552,
         'er': 2527,
 

In [58]:
word_counts_in_processed_sentences['<UNK>']

61020

In [37]:
ngrams = generate_ngrams(processed_sentences, max_len=5)

# Sample output:
for i in range(5):
    print(ngrams[i])

(['homa'], 'mkhbbyin')
(['homa', 'mkhbbyin'], 'chi')
(['homa', 'mkhbbyin', 'chi'], 'haja')
(['homa', 'mkhbbyin', 'chi', 'haja'], 'ana')
(['homa', 'mkhbbyin', 'chi', 'haja', 'ana'], 'mti99en')


In [38]:
sequences = [([word2idx[word] for word in context], word2idx[target]) for context, target in ngrams]

In [39]:
sequences

[([12490], 28783),
 ([12490, 28783], 7161),
 ([12490, 28783, 7161], 12095),
 ([12490, 28783, 7161, 12095], 4152),
 ([12490, 28783, 7161, 12095, 4152], 29943),
 ([5092], 12490),
 ([5092, 12490], 38547),
 ([5092, 12490, 38547], 12754),
 ([5092, 12490, 38547, 12754], 27987),
 ([23176], 24699),
 ([23176, 24699], 9830),
 ([23176, 24699, 9830], 29298),
 ([23176, 24699, 9830, 29298], 6548),
 ([11220], 11635),
 ([11220, 11635], 971),
 ([11220, 11635, 971], 28915),
 ([11220, 11635, 971, 28915], 21933),
 ([37938], 34378),
 ([37938, 34378], 28845),
 ([38563], 11381),
 ([4328], 21381),
 ([4328, 21381], 7774),
 ([4328, 21381, 7774], 35255),
 ([18057], 8443),
 ([18057, 8443], 440),
 ([18057, 8443, 440], 4263),
 ([18057, 8443, 440, 4263], 5451),
 ([18057, 8443, 440, 4263, 5451], 32321),
 ([8443, 440, 4263, 5451, 32321], 5465),
 ([440, 4263, 5451, 32321, 5465], 30319),
 ([4263, 5451, 32321, 5465, 30319], 6638),
 ([5451, 32321, 5465, 30319, 6638], 4797),
 ([32321, 5465, 30319, 6638, 4797], 21381),
 ([5

In [40]:
def left_pad_sequence(seq, max_len):
    return [0] * (max_len - len(seq)) + seq


In [41]:
import torch

# 1. Left pad function
def left_pad_sequence(seq, max_len):
    return torch.tensor([0] * (max_len - len(seq)) + seq.tolist())

# 2. Prepare inputs and targets
inputs = [torch.tensor(seq[0]) for seq in sequences]
targets = [seq[1] for seq in sequences]

# 3. Pad all inputs to same length (e.g., max_len = 5)
max_len = 5
padded_inputs = torch.stack([left_pad_sequence(seq, max_len) for seq in inputs])

# 4. Convert targets to tensor
targets = torch.tensor(targets)


In [42]:
padded_inputs

tensor([[    0,     0,     0,     0, 12490],
        [    0,     0,     0, 12490, 28783],
        [    0,     0, 12490, 28783,  7161],
        ...,
        [37020, 23117, 11175, 33467, 20895],
        [23117, 11175, 33467, 20895, 12581],
        [11175, 33467, 20895, 12581, 34315]])

In [43]:
from torch.utils.data import Dataset, DataLoader

class NgramDataset(Dataset):
    def __init__(self, padded_inputs, targets):
        self.padded_inputs = padded_inputs
        self.targets = targets

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.padded_inputs[idx], self.targets[idx]

In [44]:
dataset = NgramDataset(padded_inputs, targets)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NextWordPredictor(nn.Module):
    def __init__(self, vocab_size, embedding_dim=50, hidden_size=128, num_layers=2):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.bi_lstm = nn.LSTM(input_size=embedding_dim,
                               hidden_size=hidden_size,
                               num_layers=num_layers,
                               bidirectional=True,
                               batch_first=True)
        self.fc = nn.Linear(hidden_size * 2, vocab_size)  # *2 for bidirectional

    def forward(self, x):
        # x: (batch_size, sequence_length)
        embedded = self.embedding(x)  # (batch_size, seq_len, embedding_dim)
        lstm_out, _ = self.bi_lstm(embedded)  # (batch_size, seq_len, hidden_size*2)
        last_hidden = lstm_out[:, -1, :]  # take output of the last time step
        output = self.fc(last_hidden)  # (batch_size, vocab_size)
        return output


In [46]:
model = NextWordPredictor(vocab_size=len(word2idx))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [47]:
len(dataset)

213553

In [48]:
from tqdm import tqdm  # Make sure to install tqdm if not already: pip install tqdm

num_epochs = 1

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    correct = 0
    total = 0

    loop = tqdm(dataloader, desc=f"Epoch [{epoch+1}/{num_epochs}]", leave=False)

    for inputs, targets in loop:
        inputs, targets = inputs.to(device), targets.to(device)

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, targets)
        epoch_loss += loss.item()

        # Backward pass + optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accuracy
        _, predicted = torch.max(outputs, dim=1)
        correct += (predicted == targets).sum().item()
        total += targets.size(0)

        # Update tqdm progress bar with loss and accuracy info
        loop.set_postfix(loss=loss.item(), acc=(correct / total) * 100)

    avg_loss = epoch_loss / len(dataloader)
    accuracy = correct / total * 100

    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {avg_loss:.4f} - Accuracy: {accuracy:.2f}%")


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn.functional as F

def predict_next_word(model, input_seq, word2idx, idx2word, max_len=5, device='cpu'):
    model.eval()
    
    # Tokenize & numericalize input_seq (a list of tokens or token IDs)
    # If input_seq is words, convert to indices:
    if isinstance(input_seq[0], str):
        input_seq = [word2idx.get(w, word2idx['<UNK>']) for w in input_seq]

    # Left pad to max_len
    input_tensor = torch.tensor([0] * (max_len - len(input_seq)) + input_seq).unsqueeze(0).to(device)
    
    with torch.no_grad():
        outputs = model(input_tensor)  # (1, vocab_size)
        probs = F.softmax(outputs, dim=1)
        predicted_idx = torch.argmax(probs, dim=1).item()

    predicted_word = idx2word.get(predicted_idx, '<UNK>')
    return predicted_word


In [ ]:
input_seq = "mal haitam tay bghi"
predict_next_word(model, input_seq.split(), word2idx, idx2word, max_len=5, device=device)

'yosra'